In [15]:
import pandas as pd
import mysql.connector as conn


In [16]:
# We must import the password, in my case I'd like to keep it private, so I included it in a file .txt and will import it from there,
password = open ("../passwords.txt").readlines()[0]
password = password.strip()

In [17]:
crea_db=conn.connect(host='localhost',
                     user='root',
                     passwd= password )

cursor=crea_db.cursor()
# check
cursor.execute('show databases')
if 'publications' in cursor: print('ok')

In [18]:
db=conn.connect(host='localhost', # We can now connect to publications
                     user='root',  
                     passwd=password,
                     database='publications')

cursor=db.cursor()
cursor.execute('show tables') # we can check that we are in the right database
# for x in cursor: # We checked and we are in the right database
#     print(x)
tablas = [i[0] for i in cursor] # we safe all the tables in a list to access easly

In [19]:
# Now we can already write the right queries in order to work on the challange,
# We need to use the formula, sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
# let's write and execute the first query,
# STEP 1, once it works I am going to comment it 
# cursor.execute(''' select t.title_id as 'Title ID', a.au_id as 'Authors ID', (t.price*qty*t.royalty/100*ta.royaltyper/100) as 'Royalty'
#                     from authors as a 
#                     left join titleauthor as ta on ta.au_id = a.au_id
#                     left join titles as t on t.title_id = ta.title_id
#                     left join sales as s on s.title_id = t.title_id
#                     ;''')
# Step 2
# cursor.execute(''' select t.title_id as 'Title ID', a.au_id as 'Authors ID', SUM(t.price*qty*t.royalty/100*ta.royaltyper/100) as 'SUM Royalty'
#                     from authors as a
#                     left join titleauthor as ta 
#                     on ta.au_id = a.au_id
#                     left join titles as t
#                     on t.title_id = ta.title_id
#                     inner join sales as s 
#                     on s.title_id = t.title_id
#                     group by t.title_id, a.au_id
#                     ; ''')
# Step 3
cursor.execute('''select a.au_id as 'Authors ID', SUM(t.price*qty*t.royalty/100*ta.royaltyper/100) as 'SUM Royalty'
                    from authors as a
                    left join titleauthor as ta 
                    on ta.au_id = a.au_id
                    left join titles as t
                    on t.title_id = ta.title_id
                    inner join sales as s 
                    on s.title_id = t.title_id
                    group by a.au_id
                    ;''')
                

In [21]:
db=conn.connect(host='localhost', # We can now connect to publications
                     user='root',  
                     passwd=password,
                     database='publications')
cursor=db.cursor()
# Another way to do this is using temporaty tables, and then using the previous temporary tables to generate a new output
# STEP 1, once it works I am going to comment it 
cursor.execute('''DROP TABLE IF EXISTS Step1Table
;

CREATE temporary table Step1Table
select t.title_id as 'Title_ID', a.au_id as 'Authors_ID', (t.price*qty*t.royalty/100*ta.royaltyper/100) as 'Royalty'
from authors as a
left join titleauthor as ta 
on ta.au_id = a.au_id
left join titles as t
on t.title_id = ta.title_id
inner join sales as s 
on s.title_id = t.title_id
;
DROP TABLE IF EXISTS Step2Table
;
CREATE temporary table Step2Table
select Title_ID,Authors_ID, SUM(Royalty) 
from Step1Table 
group by Title_ID, Authors_ID
;

select Authors_ID, SUM(Royalty) 
from Step1Table 
group by Authors_ID
;''')
# The result from this challange and the previous challange is exacly the same!!!!


In [ ]:
db=conn.connect(host='localhost', # We can now connect to publications
                     user='root',  
                     passwd=password,
                     database='publications')
cursor=db.cursor()
# For this next challange I am using an additonal way of generating this data, with derivative tables
cursor.execute('''create table most_profiting_authors 
select Authors_ID, SUM(Royalty) most_profiting_authors
from (select Title_ID, Authors_ID, SUM(Royalty)  as 'Royalty'
from (select t.title_id as 'Title_ID', a.au_id as 'Authors_ID', (t.price*qty*t.royalty/100*ta.royaltyper/100) as 'Royalty'
from authors as a
left join titleauthor as ta 
on ta.au_id = a.au_id
left join titles as t
on t.title_id = ta.title_id
inner join sales as s 
on s.title_id = t.title_id) myDerivedTable
group by Title_ID, Authors_ID) mdt
group by Authors_ID
;
select * from most_profiting_authors ''')
# The result from this challange and the previous challange is exacly the same!!!!
